# 用 Python 脚本调用 DeepL API Pro 进电子书的行自动翻译

## 概述

![](images/deepl-api-python.gif)

## 1. 电子书格式转换路径

首先，需要将电子书从 Kindle 中导出来，并用 ePubor 进行 deDRM，而后将电子书转换成 epub 文件。

随后，再用 Calibre 将 epub 文件转换成 `htmlz` 压缩包。

在 Terminal 里用 unzip 命令解开 `htmlz` 压缩包。

## 2. 选择 html 格式作为翻译格式的原因

1. 可以保留书中大量的脚注、尾注及其链接；
2. DeepL 有专门的 API 参数处理 xml tag，`tag_handling="xml"`；
3. 可以通过 css 文件随意设置显示样式，比较灵活；
4. 可以通过插入 javascript 函数指定某种特定语言的显示（比如，只显示中文）；
5. 可以用来作为源文件转换成任意格式的电子书……

另外，在调用 `tag_handling="xml"` 之后，DeepL API 返回的译文非常规整，能够保留所有 html tag；并且，“返回字符串” 与 “原字符串” 相同，可以作为一个判断依据 —— 该行有没有被翻译，如果没有，在生成的译文 html 文件中，该行没必要重复出现……

## 3. 清理 html

html 文件整理起来比较麻烦，一个比较方便的手段是使用 `BeautifulSoup` 模块。`BeautifulSoup` 本来是爬虫工具，但，它又很方便的手段可以清理 html 文件。

以下脚本主要完成以下工作：

* 首先将 html 文件里的所有 `\n` 去掉；
* 将所有 `<div>` 单独放在一行；
* 将所有 `</div>` 也单独放在一行；
* 将 `<p>` 内部的所有 `\n` 全都去掉；并在之前加上一个空行；
* …… 当然，你可以在这里做更多你自己喜欢做的格式清理。

为了方便起见，`path` 和 `source_filename` 以及 `target_filename` 都单独指定。

In [ ]:
import bs4
import re

path = "<your files folder name with / as end>" # 文件夹名称末尾得有 /
source_filename = "index.html"
target_filename = "index2.html"

html = open(path+source_filename)
htmltext = html.read()

soup = bs4.BeautifulSoup(htmltext)

# 将所有的 \n 去掉……
htmltext = str(bs4.BeautifulSoup(htmltext)).replace("\n", "")

# <div... 之前添加空行
pttn = r'<div'
rpl = r'\n\n<div'
re.findall(pttn, htmltext)
htmltext = re.sub(pttn, rpl, htmltext)

# </div> 之前添加空行
pttn = r'</div>'
rpl = r'\n\n</div>'
re.findall(pttn, htmltext)
htmltext = re.sub(pttn, rpl, htmltext)

# <p... 之前添加空行
pttn = r'<p'
rpl = r'\n\n<p'
re.findall(pttn, htmltext)
htmltext = re.sub(pttn, rpl, htmltext)

fileSave = open(path+target_filename, "w")
fileSave.write(htmltext)
print(htmltext)

## 4. 逐行提交 DeepL API Pro 进行翻译

将清理过的 html 交给以下脚本，逐行提交给 DeepL 翻译，并返回。

为了方便起见，`path` 和 `source_filename` 以及 `target_filename` 都单独指定。

* `lines` 是 `source_filename` 的内容
* `new_lines` 是将要放到 `target_filename` 中的内容
* `startline` 是 “从哪一行开始提交 DeepL 翻译”
* `endline` 是 “到哪一行开始结束提交 DeepL 翻译”

In [ ]:
import re
import requests

auth_key = "<your DeepL API Pro authentication key>" # 注意，要订阅的是 DeepL API Pro
target_language = "ZH"  ## 当然，你可以将目标语言设置成任何 DeepL 支持的语言

path = "<your files folder name with / as end>" # 文件夹名称末尾得有 /
source_filename = "index2.html" # 上一步生成的文件，成为这一步的 “源文件”
target_filename = "index3.html"

def translate(text):
    result = requests.get( 
       "https://api.deepl.com/v2/translate",
       params={ 
         "auth_key": auth_key,
         "target_lang": target_language,
         "text": text,
         "tag_handling": "xml", # 这个参数确保 DeepL 正确处理 html tags
       },
    ) 
    return result.json()["translations"][0]["text"]

def add_language_tag_en(html):
    pttn = re.compile(r'^<(.*?) class="(.*?)">', re.M)
    rpl = r'<\1 class="\2 en">'
    re.findall(pttn, html)
    html = re.sub(pttn, rpl, html)
    return html

def add_language_tag_cn(html):
    pttn = re.compile(r'^<(.*?) class="(.*?)">', re.M)
    rpl = r'<\1 class="\2 cn">'
    re.findall(pttn, html)
    html = re.sub(pttn, rpl, html)
    return html

lines = open(path+source_filename, "r").readlines()


new_lines = []
line_count = 0
startline = 16
endline = 4032

for line in lines:
    line_count += 1
    if line_count < startline or line_count > endline or line.strip() == '':
        new_lines.append(line)
        print(line)
        continue        
    
    succeeded = False
    while not succeeded:
        # 以下比较粗暴的 try... except，用来防止执行过程中出现 DeepL 连接错误而导致翻译任务中断……
        try:
            line_translated = translate(line)
            # 以下一行确保将返回的字符串转换成一整行，而非含有 \n 的多行文本
            line_translated = line_translated.replace("\n", "")
            
            succeeded = True
        except:
            succeeded = False
    
    if line.strip() == line_translated.strip(): 
        #返回的字符串与原字符串相同，说明 html tag 之间的内容无需翻译
        new_lines.append(line)
        print(line)
    else:
        line = add_language_tag_en(line)
        line_translated = add_language_tag_cn(line_translated)
        new_lines.append(line)
        print(line)
        new_lines.append(line_translated)
        print(line_translated)

with open(path+target_filename, 'w') as f:
    f.write("\n".join(new_lines))

## 5. 对翻译好的中文文本进行自动排版

有很多细节，比如：

* `"` 要相应地替换成 `“”`
* `'` 要相应地替换成 `‘’`
* 无论是单引号还是双引号，都要与相邻的字符之间留有一个空格；标点符号 `，。？！` 除外；
* 数字、百分比、英文字母，与汉字之间应该应该留有空格；
* 应用在中文字符的 “斜体” 样式，应该改成 “加重” 样式；
* 破折号统一使用 `——`（前后有空格，除非与标点符号相邻）而非 `&mdash;` 或者 `--`；
* 外国姓名之间的符号为 `·`；
* ……

In [2]:
import re

def zh_format(html):
    
    # 去掉半角方括号
    pttn = r'\[(.*?)\]'
    rpl = r'\1'
    re.findall(pttn, html)
    html = re.sub(pttn, rpl, html)
    
    # 直双引号转换成弯双引号
    pttn = r'\s*"(.*?)\s*"'
    rpl = r'“\1”'
    re.findall(pttn, html)
    html = re.sub(pttn, rpl, html)
    
    # 直单引号转换成弯单引号
    pttn = r"\s*'(.*?)\s*'"
    rpl = r'‘\1’'
    re.findall(pttn, html)
    html = re.sub(pttn, rpl, html)
    
    # html tag 中被误伤的直引号
    pttn = r'=[“”"](.*?)[“”"]'
    rpl = r'="\1"'
    re.findall(pttn, html)
    html = re.sub(pttn, rpl, html)
    
    # html 弯引号之前的空格
    pttn = r'([\u4e00-\u9fa5])([“‘])'
    rpl = r'\1 \2'
    re.findall(pttn, html)
    html = re.sub(pttn, rpl, html)
    
    # html 弯引号之后的空格
    pttn = r'([’”])([\u4e00-\u9fa5])'
    rpl = r'\1 \2'
    re.findall(pttn, html)
    html = re.sub(pttn, rpl, html)
           
    # html tag: <i>, <em> 转换成 <strong>
    pttn = r'(<i|<em)'
    rpl = r'<strong'
    re.findall(pttn, html)
    html = re.sub(pttn, rpl, html)
    
    # html tag: <i>, <em> 转换成 <strong>
    pttn = r'(i>|em>)'
    rpl = r'strong>'
    re.findall(pttn, html)
    html = re.sub(pttn, rpl, html)
    
    # html tag: strong 内部的 “”、‘’、《》、（）
    pttn = r'([《（“‘]+)<strong (.*?)>'
    rpl = r'<strong \2>\1'
    re.findall(pttn, html)
    html = re.sub(pttn, rpl, html)
    
    pttn = r'</strong>([》）”’]+)'
    rpl = r'\1</strong>'
    re.findall(pttn, html)
    html = re.sub(pttn, rpl, html)
    
    # 省略号
    pttn = r'\.{2,}\s*。*\s*'
    rpl = r'…… '
    re.findall(pttn, html)
    html = re.sub(pttn, rpl, html)   
    
    # 破折号
    pttn = r'&mdash；|&mdash;|--'
    rpl = r' —— '
    re.findall(pttn, html)
    html = re.sub(pttn, rpl, html)
    
    # 姓名之间的 ·（重复三次）
    pttn = r'([\u4e00-\u9fa5])-([\u4e00-\u9fa5])'
    rpl = r'\1·\2'
    re.findall(pttn, html)
    html = re.sub(pttn, rpl, html)
    
    pttn = r'([\u4e00-\u9fa5])-([\u4e00-\u9fa5])'
    rpl = r'\1·\2'
    re.findall(pttn, html)
    html = re.sub(pttn, rpl, html)
    
    pttn = r'([\u4e00-\u9fa5])-([\u4e00-\u9fa5])'
    rpl = r'\1·\2'
    re.findall(pttn, html)
    html = re.sub(pttn, rpl, html)
    
    pttn = r'([A-Z]{1})\s*\.\s*([A-Z]{1})'
    rpl = r'\1·\2'
    re.findall(pttn, html)
    html = re.sub(pttn, rpl, html)    
    
    pttn = r'([A-Z]{1})\s*\.\s*([\u4e00-\u9fa5])'
    rpl = r'\1·\2'
    re.findall(pttn, html)
    html = re.sub(pttn, rpl, html)

    # 全角百分号
    pttn = r'％'
    rpl = r'%'
    re.findall(pttn, html)
    html = re.sub(pttn, rpl, html)
      
    # 数字前的空格
    pttn = r'([\u4e00-\u9fa5])(\d)'
    rpl = r'\1 \2'
    re.findall(pttn, html)
    html = re.sub(pttn, rpl, html)
    
    # 数字后的空格，百分比 % 后的空格
    pttn = r'([\d%])([\u4e00-\u9fa5])'
    rpl = r'\1 \2'
    re.findall(pttn, html)
    html = re.sub(pttn, rpl, html)
        
    # 英文字母前的空格
    pttn = r'([\u4e00-\u9fa5])([a-zA-Z])'
    rpl = r'\1 \2'
    re.findall(pttn, html)
    html = re.sub(pttn, rpl, html)
        
    # 英文字母后的空格，百分比 % 后的空格
    pttn = r'([a-zA-Z])([\u4e00-\u9fa5])'
    rpl = r'\1 \2'
    re.findall(pttn, html)
    html = re.sub(pttn, rpl, html)
        
    # tag 内的英文字母前的空格
    pttn = r'([\u4e00-\u9fa5])<(strong|i|em|span)>(.[a-zA-Z\d ]*?)<\/(strong|i|em|span)>'
    rpl = r'\1 <\2>\3</\4>'
    re.findall(pttn, html)
    html = re.sub(pttn, rpl, html)
        
    # tag 内的英文字母后的空格，百分比 % 后的空格
    pttn = r'<(strong|i|em|span)>(.[a-zA-Z\d ]*?)<\/(strong|i|em|span)>([\u4e00-\u9fa5])'
    rpl = r'<\1>\2</\3> \4'
    re.findall(pttn, html)
    html = re.sub(pttn, rpl, html)
    
    # 弯引号前的逗号
    pttn = r'，([”’])'
    rpl = r'\1，'
    re.findall(pttn, html)
    html = re.sub(pttn, rpl, html)
        
    # 中文标点符号之前多余的空格
    pttn = r'([，。！？》〉】]) '
    rpl = r'\1'
    re.findall(pttn, html)
    html = re.sub(pttn, rpl, html)
    
    # 英文句号 . 与汉字之间的空格
    pttn = r'\.([\u4e00-\u9fa5])'
    rpl = r'. \1'
    re.findall(pttn, html)
    html = re.sub(pttn, rpl, html)
      
    # 左半角括号
    pttn = r'\s*\('
    rpl = r'（'
    re.findall(pttn, html)
    html = re.sub(pttn, rpl, html)
    
    # 右半角括号
    pttn = r'\)\s*'
    rpl = r'）'
    re.findall(pttn, html)
    html = re.sub(pttn, rpl, html)  

    # 多余的括号（DeepL 返回文本经常出现的情况）
    pttn = r'）。）'
    rpl = r'。）'
    re.findall(pttn, html)
    html = re.sub(pttn, rpl, html)  

    # # 摄氏温度符号
    # pttn = r'℃'
    # rpl = r'\1·\2'
    # re.findall(pttn, html)
    # html = re.sub(pttn, rpl, html)
    
    return html

path = "<your files folder name with / as end>" # 文件夹名称末尾得有 /
source_filename ="index3.html"  # 上一步生成的文件，成为这一步的 “源文件”
target_filename = "index4.html"

lines = open(path+source_filename, "r").readlines()

new_lines = []
for line in lines:
    if 'cn"><img ' in line:
        continue
    if ' cn"' in line:
        new_lines.append(zh_format(line))
    else:
        new_lines.append(line)

with open(path+target_filename, 'w') as f:
    f.write("".join(new_lines))

## 6. 然后……

接下来要做的是：

* 整理一下 css，使浏览器渲染更为悦目；
  * 为 `.cn` classs 设定 `letter-spacing: 0.1em;`
  * 可以设置 `p.en` 为 `{display: none;}`，这样的话，英文就不显示了 —— 方便用 Edge 的语音朗读听书……
* 校对翻译文本，比如：
  * 有些 `<span></span>` 的位置不对；
  * 有些引号并没有配对；
  * 很多人名、地名需要统一（译文统一、格式统一）
  * 有些人名、地名没有被翻译过来
  * 少数地方可能不符合之前定义的中文排版约定（比如，多个空格少个空格什么的）
  * 经常有些地方，DeepL 可能翻译 “反了”
  * 有些引用文字，由于被 “ xxx 说“，之类的插入语切段，所以，翻译或者错了，或者重复
  
